In [ ]:
### feature 대상으로 LGBM + CatBoost + XGBoost Ensemble Voting 학습
# (Developed from dacon_etri_base_mod4_1_6.ipynb)

from datetime import datetime

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530

submission_folder = '/users/KTL/Desktop/dacon/submission/'
submission_file = f'submission_ensemble_voting_final_mod4_1_6_{timestamp}.csv'


In [ ]:
# =========================== 라이브러리 임포트 ===========================

import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 

# 앙상블 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import torch
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
from functools import reduce
import math

In [ ]:
# =========================== 시드 고정 ===========================

# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

In [ ]:
# =========================== 파일 경로 설정 ===========================

# 파일 경로 설정 - VSCode 상대경로로 변경
base_folder =  '/users/KTL/Desktop/ETRI_lifelog_dataset'
folder = '/ch2025_data_items'

data_dir = base_folder + folder 

# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 

In [ ]:
# =========================== 데이터 로딩 ===========================

# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 

# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

In [ ]:
# =========================== FEATURE ENGINEERING ===========================

def process_mACStatus(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df = df.sort_values(['subject_id', 'timestamp']) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_charging'].values # 0/1 상태 
        times = group['timestamp'].values # 충전 상태 비율 
        ratio_charging = status.mean() 
        # 상태 전이 횟수 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        lengths = [] 
        current_len = 0 
        for val in status: 
            if val == 1: 
                current_len += 1 
            elif current_len > 0: 
                lengths.append(current_len) 
                current_len = 0 
        if current_len > 0: 
            lengths.append(current_len) 
        avg_charging_duration = np.mean(lengths) if lengths else 0 
        max_charging_duration = np.max(lengths) if lengths else 0 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'charging_ratio': ratio_charging, 
            'charging_transitions': transitions, 
            'avg_charging_duration': avg_charging_duration, 
            'max_charging_duration': max_charging_duration, 
        }) 
    return pd.DataFrame(results) 

def process_mActivity(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        counts = group['m_activity'].value_counts(normalize=True) # 비율 
        row = {'subject_id': subj, 'date': date} 
        # 0~8 비율 저장 
        for i in range(9): 
            row[f'activity_{i}_ratio'] = counts.get(i, 0) 
        # 주요 활동 정보 
        row['dominant_activity'] = group['m_activity'].mode()[0] 
        row['num_unique_activities'] = group['m_activity'].nunique() 
        summary.append(row) 
    return pd.DataFrame(summary) 

# 지정된 10개 라벨 
top_10_labels = [ 
    "Inside, small room", "Speech", "Silence", "Music", "Narration, monologue", 
    "Child speech, kid speaking", "Conversation", "Speech synthesizer", "Shout", "Babbling" 
] 

def process_mAmbience_top10(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    # 초기화 
    for label in top_10_labels + ['others']: 
        df[label] = 0.0 
    for idx, row in df.iterrows(): 
        parsed = ast.literal_eval(row['m_ambience']) if isinstance(row['m_ambience'], str) else row['m_ambience'] 
        others_prob = 0.0 
        for label, prob in parsed: 
            prob = float(prob) 
            if label in top_10_labels: 
                df.at[idx, label] = prob 
            else: 
                others_prob += prob 
        df.at[idx, 'others'] = others_prob 
    return df.drop(columns=['m_ambience']) 

def summarize_mAmbience_daily(df): 
    prob_cols = [col for col in df.columns if col not in ['subject_id', 'timestamp', 'date']] 
    # 하루 단위로 평균값 요약 
    daily_summary = df.groupby(['subject_id', 'date'])[prob_cols].mean().reset_index() 
    return daily_summary 

def process_mBle(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble'] 
        rssi_list = [] 
        class_0_cnt = 0 
        class_other_cnt = 0 
        for device in entry: 
            try: 
                rssi = int(device['rssi']) 
                rssi_list.append(rssi) 
                if str(device['device_class']) == '0': 
                    class_0_cnt += 1 
                else: 
                    class_other_cnt += 1 
            except: 
                continue # malformed record 
        feature = { 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'device_class_0_cnt': class_0_cnt, 
            'device_class_others_cnt': class_other_cnt, 
            'device_count': len(rssi_list), 
            'rssi_mean': np.mean(rssi_list) if rssi_list else np.nan, 
            'rssi_min': np.min(rssi_list) if rssi_list else np.nan, 
            'rssi_max': np.max(rssi_list) if rssi_list else np.nan, 
        } 
        features.append(feature) 
    return pd.DataFrame(features) 

def summarize_mBle_daily(df): 
    # row 단위 BLE feature 추출 
    df = process_mBle(df) 
    # 하루 단위로 cnt 합치기 
    grouped = df.groupby(['subject_id', 'date']).agg({ 
        'device_class_0_cnt': 'sum', 
        'device_class_others_cnt': 'sum', 
        'rssi_mean': 'mean', 
        'rssi_min': 'min', 
        'rssi_max': 'max', 
    }).reset_index() 
    # 총합 구해서 비율 계산 
    total_cnt = grouped['device_class_0_cnt'] + grouped['device_class_others_cnt'] 
    grouped['device_class_0_ratio'] = grouped['device_class_0_cnt'] / total_cnt.replace(0, np.nan) 
    grouped['device_class_others_ratio'] = grouped['device_class_others_cnt'] / total_cnt.replace(0, np.nan) 
    # 필요 없는 원래 cnt 컬럼 제거 
    grouped.drop(columns=['device_class_0_cnt', 'device_class_others_cnt'], inplace=True) 
    return grouped 

def process_mGps(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        gps_list = ast.literal_eval(row['m_gps']) if isinstance(row['m_gps'], str) else row['m_gps'] 
        altitudes = [] 
        latitudes = [] 
        longitudes = [] 
        speeds = [] 
        for entry in gps_list: 
            try: 
                altitudes.append(float(entry['altitude'])) 
                latitudes.append(float(entry['latitude'])) 
                longitudes.append(float(entry['longitude'])) 
                speeds.append(float(entry['speed'])) 
            except: 
                continue 
        features.append({ 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'altitude_mean': np.mean(altitudes) if altitudes else np.nan, 
            'latitude_std': np.std(latitudes) if latitudes else np.nan, 
            'longitude_std': np.std(longitudes) if longitudes else np.nan, 
            'speed_mean': np.mean(speeds) if speeds else np.nan, 
            'speed_max': np.max(speeds) if speeds else np.nan, 
            'speed_std': np.std(speeds) if speeds else np.nan, 
        }) 
    return pd.DataFrame(features) 

def process_mLight(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['hour'] = df['timestamp'].dt.hour 
    # 밤(22~05시), 낮(06~21시) 구분 
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6) 
    # 하루 단위 요약 
    daily = df.groupby(['subject_id', 'date']).agg( 
        light_mean=('m_light', 'mean'), 
        light_std=('m_light', 'std'), 
        light_max=('m_light', 'max'), 
        light_min=('m_light', 'min'), 
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()), 
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()), 
        light_night_ratio=('is_night', 'mean') # 밤 시간 측정 비율 
    ).reset_index() 
    return daily 

def process_mScreenStatus(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_screen_use'].values 
        ratio_on = status.mean() 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        durations = [] 
        current = 0 
        for val in status: 
            if val == 1: 
                current += 1 
            elif current > 0: 
                durations.append(current) 
                current = 0 
        if current > 0: 
            durations.append(current) 
        features.append({ 
            'subject_id': subj, 
            'date': date, 
            'screen_on_ratio': ratio_on, 
            'screen_on_transitions': transitions, 
            'screen_on_duration_avg': np.mean(durations) if durations else 0, 
            'screen_on_duration_max': np.max(durations) if durations else 0, 
        }) 
    return pd.DataFrame(features) 

top_apps = [ 
    'One UI 홈', '카카오톡', '시스템 UI', 'NAVER', '캐시워크', 
    '성경일독Q', 'YouTube', '통화', '메시지', '타임스프레드', 'Instagram'
] 

def process_mUsageStats(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        app_time = {app: 0 for app in top_apps} 
        others_time = 0 
        for row in group['m_usage_stats']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for entry in parsed: 
                app = entry.get('app_name') 
                time = entry.get('total_time', 0) 
                if app in top_apps: 
                    app_time[app] += int(time) 
                else: 
                    others_time += int(time) 
        feature = { 
            'subject_id': subj, 
            'date': date, 
            'others_time': others_time 
        } 
        # 각 앱별 컬럼 추가 
        feature.update({f'{app}_time': app_time[app] for app in top_apps}) 
        features.append(feature) 
    return pd.DataFrame(features) 

def process_mWifi(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        rssi_all = [] 
        for row in group['m_wifi']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for ap in parsed: 
                try: 
                    rssi = int(ap['rssi']) 
                    rssi_all.append(rssi) 
                except: 
                    continue 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'wifi_rssi_mean': np.mean(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_min': np.min(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_max': np.max(rssi_all) if rssi_all else np.nan, 
            'wifi_detected_cnt': len(rssi_all) 
        }) 
    return pd.DataFrame(results) 

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

def process_wLight_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            lux = block_group['w_light'].dropna().values 
            if len(lux) == 0: 
                continue 
            block_stats[f'wlight_{block}_mean'] = np.mean(lux) 
            block_stats[f'wlight_{block}_std'] = np.std(lux) 
            block_stats[f'wlight_{block}_max'] = np.max(lux) 
            block_stats[f'wlight_{block}_min'] = np.min(lux) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

def process_wPedo(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = df.groupby(['subject_id', 'date']).agg({ 
        'step': 'sum', 
        'step_frequency': 'mean', 
        'distance': 'sum', 
        'speed': ['mean', 'max'], 
        'burned_calories': 'sum' 
    }).reset_index() 
    # 컬럼 이름 정리 
    summary.columns = ['subject_id', 'date', 'step_sum', 'step_frequency_mean', 'distance_sum', 'speed_mean', 'speed_max', 'burned_calories_sum'] 
    return summary 

# 모든 feature processing 실행
print("=== Feature Engineering 시작 ===")

mACStatus_df2 = process_mACStatus(mACStatus_df)
print("✅ mACStatus 처리 완료")

mActivity_df2 = process_mActivity(mActivity_df)
print("✅ mActivity 처리 완료")

mAmbience_df2= process_mAmbience_top10(mAmbience_df)
mAmbience_df2 = summarize_mAmbience_daily(mAmbience_df2)
print("✅ mAmbience 처리 완료")

mBle_df2 = summarize_mBle_daily(mBle_df)
print("✅ mBle 처리 완료")

m_Gps_df2 = process_mGps(mGps_df)
m_Gps_df2 = m_Gps_df2.groupby(['subject_id', 'date']).agg({
    'altitude_mean': 'mean',
    'latitude_std': 'mean',
    'longitude_std': 'mean',
    'speed_mean': 'mean',
    'speed_max': 'max',
    'speed_std': 'mean'
}).reset_index()
print("✅ mGps 처리 완료")

mLight_df2 = process_mLight(mLight_df)
print("✅ mLight 처리 완료")

mScreenStatus_df2 = process_mScreenStatus(mScreenStatus_df)
print("✅ mScreenStatus 처리 완료")

mUsageStats_df2 = process_mUsageStats(mUsageStats_df)
print("✅ mUsageStats 처리 완료")

mWifi_df2 = process_mWifi(mWifi_df)
print("✅ mWifi 처리 완료")

wHr_df2 = process_wHr_by_timeblock(wHr_df)
print("✅ wHr 처리 완료")

wLight_df2 = process_wLight_by_timeblock(wLight_df)
print("✅ wLight 처리 완료")

wPedo_df2 = process_wPedo(wPedo_df)
print("✅ wPedo 처리 완료")

In [ ]:
# =========================== 데이터 병합 ===========================

df_list = [ 
    mACStatus_df2, 
    mActivity_df2,
    mAmbience_df2, 
    mBle_df2, 
    m_Gps_df2, 
    mLight_df2, 
    mScreenStatus_df2, 
    mUsageStats_df2, 
    mWifi_df2, 
    wHr_df2, 
    wLight_df2, 
    wPedo_df2 
] 

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list) 

# metrics_train의 lifelog_date → datetime.date 형으로 변환 
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date 

# merged_df의 date도 변환 
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date 

# 1. date 기준 정렬을 위해 metrics_train의 lifelog_date -> date로 맞추기 
metrics_train_renamed = metrics_train.rename(columns={'lifelog_date': 'date'}) 

# 2. train_df: metrics_train과 일치하는 (subject_id, date) → 라벨 포함 
train_df = pd.merge(metrics_train_renamed, merged_df, on=['subject_id', 'date'], how='inner') 

# 3. test_df: metrics_train에 없는 (subject_id, date) 
merged_keys = merged_df[['subject_id', 'date']] 
train_keys = metrics_train_renamed[['subject_id', 'date']] 
test_keys = pd.merge(merged_keys, train_keys, on=['subject_id', 'date'], how='left', indicator=True) 
test_keys = test_keys[test_keys['_merge'] == 'left_only'].drop(columns=['_merge']) 
test_df = pd.merge(test_keys, merged_df, on=['subject_id', 'date'], how='left') 

print(f"✅ 데이터 병합 완료: train_df={train_df.shape}, test_df={test_df.shape}")

In [ ]:
# =========================== 데이터 전처리 ===========================

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

# ✅ feature 준비 
X = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']) 
X.fillna(0, inplace=True) # 결측값 처리 

test_X = test_df.drop(columns=['subject_id', 'date']) 
test_X.fillna(0, inplace=True) 

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True) # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True) # 연속된 _ 제거 
        .str.strip("_") # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X = sanitize_column_names(X) 
test_X = sanitize_column_names(test_X) 

# 모든 특성에 대해 모델링 진행 
X_selected = X.copy()
test_X_selected = test_X.copy()

print(f"✅ 전처리 완료: X_selected={X_selected.shape}, test_X_selected={test_X_selected.shape}")

In [ ]:
# =========================== 기본 LGBM으로 Feature Importance 구하기 ===========================

print("=== 기본 Feature Importance 추출 ===")

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

# 이진 분류
binary_models_selected = {}
for col in targets_binary:
    print(f">> {col} 변수 Feature Importance 추출 중..")
    y = train_df[col]
    binary_params = lgbm_params.copy()
    binary_params['objective'] = 'binary'
    
    model = LGBMClassifier(**binary_params)
    model.fit(X_selected, y)
    binary_models_selected[col] = model

# 다중 분류 (S1)
print(">> S1 변수 Feature Importance 추출 중..")
y_multi = train_df['S1']
multiclass_params = lgbm_params.copy()
multiclass_params['num_class'] = len(y_multi.unique())

model_s1_selected = LGBMClassifier(**multiclass_params)
model_s1_selected.fit(X_selected, y_multi)

# Feature Importance 추출
feature_importance_dict = {}

# 이진분류 모델들의 feature importance
for col_name, model in binary_models_selected.items():
    importance_df = pd.DataFrame({
        'feature': X_selected.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(10)  # 상위 10개
    feature_importance_dict[col_name] = importance_df.copy()

# S1 다중분류 모델의 feature importance
fi_multi_20 = pd.DataFrame({
    'feature': X_selected.columns,
    'importance': model_s1_selected.feature_importances_
}).sort_values('importance', ascending=False).head(20)
feature_importance_dict['S1'] = fi_multi_20.copy()

print("✅ Feature Importance 추출 완료")

In [ ]:
# =========================== 앙상블 모델 설정 ===========================

print("=== 앙상블 모델 파라미터 설정 ===")

# 교차 검증 설정
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # 시간 단축을 위해 3-fold

# 각 모델별 파라미터 그리드 (간소화하여 학습 시간 단축)
lgbm_param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.03, 0.1],
    'num_leaves': [31, 50],
    'max_depth': [-1, 6],
    'min_child_samples': [10, 20],
}

catboost_param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.03, 0.1],
    'depth': [4, 6],
    'min_data_in_leaf': [10, 20],
    'border_count': [64, 128],
}

xgb_param_grid = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.03, 0.1],
    'max_depth': [4, 6],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

def create_base_models(is_binary=True, random_state=42):
    """기본 모델들 생성"""
    if is_binary:
        lgbm = LGBMClassifier(
            objective='binary',
            random_state=random_state,
            n_jobs=-1,
            verbosity=-1
        )
        catboost = CatBoostClassifier(
            loss_function='Logloss',
            random_state=random_state,
            verbose=False,
            thread_count=-1
        )
        xgb = XGBClassifier(
            objective='binary:logistic',
            random_state=random_state,
            n_jobs=-1,
            verbosity=0
        )
    else:  # multiclass
        lgbm = LGBMClassifier(
            objective='multiclass',
            random_state=random_state,
            n_jobs=-1,
            verbosity=-1
        )
        catboost = CatBoostClassifier(
            loss_function='MultiClass',
            random_state=random_state,
            verbose=False,
            thread_count=-1
        )
        xgb = XGBClassifier(
            objective='multi:softprob',
            random_state=random_state,
            n_jobs=-1,
            verbosity=0
        )
    
    return lgbm, catboost, xgb

def get_optimal_k_neighbors(y):
    """클래스 분포에 따른 최적 k_neighbors 계산"""
    min_class_count = min(y.value_counts())
    if min_class_count <= 3:
        return max(1, min_class_count - 1)
    elif min_class_count <= 5:
        return 3
    else:
        return 5

def train_ensemble_model(model_name, target_col, is_binary=True):
    """앙상블 모델 학습"""
    print(f"\n🎯 {model_name} 앙상블 모델로 {target_col} 타겟 예측")
    print("-" * 60)
    
    # 타겟 데이터 준비
    y = train_df[target_col]
    
    # 해당 모델의 상위 20개 중요 피처 선택
    top_features = feature_importance_dict[model_name]['feature'].tolist()
    X_model_selected = X_selected[top_features]
    test_X_model_selected = test_X_selected[top_features]
    
    print(f"   - 사용 특성 수: {X_model_selected.shape[1]}개")
    print(f"   - 훈련 샘플 수: {X_model_selected.shape[0]}개")
    print(f"   - 타겟: {target_col} ({'이진분류' if is_binary else '다중분류'})")
    print(f"   - 클래스 수: {len(y.unique())}개")
    print(f"   - 원본 클래스 분포: {dict(y.value_counts().sort_index())}")
    
    # SMOTE 적용
    k_neighbors = get_optimal_k_neighbors(y)
    print(f"   - SMOTE k_neighbors: {k_neighbors}")
    
    try:
        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_resampled, y_resampled = smote.fit_resample(X_model_selected, y)
        print(f"   - SMOTE 후 클래스 분포: {dict(pd.Series(y_resampled).value_counts().sort_index())}")
        print(f"   - 샘플 수 변화: {len(y)} → {len(y_resampled)}")
    except Exception as e:
        print(f"   ⚠️  SMOTE 적용 실패: {str(e)}")
        X_resampled, y_resampled = X_model_selected, y
        print(f"   - 원본 데이터로 진행")
    
    # 기본 모델들 생성
    lgbm, catboost, xgb = create_base_models(is_binary=is_binary)
    
    print(f"   - 각 모델 개별 최적화 중...")
    
    # 개별 모델 최적화
    best_models = {}
    
    # LightGBM 최적화
    print(f"     → LightGBM 최적화 중...")
    lgbm_grid = GridSearchCV(
        lgbm, lgbm_param_grid, 
        scoring='f1_macro', cv=cv, 
        verbose=0, n_jobs=-1
    )
    lgbm_grid.fit(X_resampled, y_resampled)
    best_models['lgbm'] = lgbm_grid.best_estimator_
    print(f"       LightGBM 최고 점수: {lgbm_grid.best_score_:.4f}")
    
    # CatBoost 최적화
    print(f"     → CatBoost 최적화 중...")
    catboost_grid = GridSearchCV(
        catboost, catboost_param_grid,
        scoring='f1_macro', cv=cv,
        verbose=0, n_jobs=-1
    )
    catboost_grid.fit(X_resampled, y_resampled)
    best_models['catboost'] = catboost_grid.best_estimator_
    print(f"       CatBoost 최고 점수: {catboost_grid.best_score_:.4f}")
    
    # XGBoost 최적화
    print(f"     → XGBoost 최적화 중...")
    xgb_grid = GridSearchCV(
        xgb, xgb_param_grid,
        scoring='f1_macro', cv=cv,
        verbose=0, n_jobs=-1
    )
    xgb_grid.fit(X_resampled, y_resampled)
    best_models['xgb'] = xgb_grid.best_estimator_
    print(f"       XGBoost 최고 점수: {xgb_grid.best_score_:.4f}")
    
    # Voting Classifier 생성 (Soft Voting)
    print(f"   - Voting Ensemble 생성 중...")
    voting_clf = VotingClassifier(
        estimators=[
            ('lgbm', best_models['lgbm']),
            ('catboost', best_models['catboost']),
            ('xgb', best_models['xgb'])
        ],
        voting='soft'  # 확률 기반 투표
    )
    
    # Voting Classifier 학습
    voting_clf.fit(X_resampled, y_resampled)
    
    # 예측 수행
    predictions = voting_clf.predict(test_X_model_selected)
    
    print(f"   ✅ 앙상블 학습 완료!")
    print(f"      - 예측 완료: {len(predictions)}개 샘플")
    print(f"      - 예측 클래스 분포: {dict(pd.Series(predictions).value_counts().sort_index())}")
    
    return {
        'ensemble_model': voting_clf,
        'individual_models': best_models,
        'predictions': predictions,
        'scores': {
            'lgbm': lgbm_grid.best_score_,
            'catboost': catboost_grid.best_score_,
            'xgb': xgb_grid.best_score_
        }
    }

In [ ]:
# =========================== 앙상블 모델 학습 실행 ===========================

print("=" * 60)
print("🚀 앙상블 모델 학습 시작 (LightGBM + CatBoost + XGBoost)")
print("=" * 60)

# 결과 저장용 딕셔너리
ensemble_results = {}
ensemble_predictions = {}

# 이진분류 모델들 학습
binary_model_names = ['Q1', 'Q2', 'Q3', 'S2', 'S3']
successful_models = []
failed_models = []

for model_name in binary_model_names:
    try:
        result = train_ensemble_model(model_name, model_name, is_binary=True)
        ensemble_results[model_name] = result
        ensemble_predictions[model_name] = result['predictions']
        successful_models.append(model_name)
    except Exception as e:
        print(f"❌ {model_name} 모델 학습 실패: {str(e)}")
        failed_models.append(model_name)

# S1 다중분류 모델 학습
try:
    s1_result = train_ensemble_model('S1', 'S1', is_binary=False)
    ensemble_results['S1'] = s1_result
    ensemble_predictions['S1'] = s1_result['predictions']
    s1_success = True
except Exception as e:
    print(f"❌ S1 다중분류 모델 학습 실패: {str(e)}")
    s1_success = False

In [ ]:
# =========================== 결과 요약 ===========================

print("\n" + "=" * 60)
print("🎉 앙상블 모델 학습 완료!")
print("=" * 60)

print(f"\n📊 학습 결과 요약:")
print(f"   🟢 성공한 이진분류 모델: {len(successful_models)}개")
for model in successful_models:
    scores = ensemble_results[model]['scores']
    print(f"      - {model}: LGBM={scores['lgbm']:.4f}, CB={scores['catboost']:.4f}, XGB={scores['xgb']:.4f}")

if failed_models:
    print(f"   🔴 실패한 이진분류 모델: {len(failed_models)}개")
    for model in failed_models:
        print(f"      - {model}")

if s1_success:
    s1_scores = ensemble_results['S1']['scores']
    print(f"   🟢 S1 다중분류 모델: 성공")
    print(f"      - S1: LGBM={s1_scores['lgbm']:.4f}, CB={s1_scores['catboost']:.4f}, XGB={s1_scores['xgb']:.4f}")
else:
    print(f"   🔴 S1 다중분류 모델: 실패")


In [ ]:
# =========================== 최종 제출 파일 생성 ===========================

print("=" * 60)
print("📋 앙상블 기반 최종 제출 파일 생성")
print("=" * 60)

# sample 기반 제출 포맷 가져오기
submission_ensemble = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_ensemble['lifelog_date'] = pd.to_datetime(submission_ensemble['lifelog_date']).dt.date

print(f"📊 제출 데이터 기본 정보:")
print(f"   - 제출 샘플 수: {len(submission_ensemble):,}개")
print(f"   - 고유 피험자 수: {submission_ensemble['subject_id'].nunique()}명")

# 예측 결과 할당
def assign_predictions_safely(df, target_col, predictions, model_type="이진분류"):
    """안전하게 예측 결과를 할당"""
    if len(df) != len(predictions):
        print(f"   ⚠️  {target_col} 길이 불일치: submission={len(df)}, predictions={len(predictions)}")
        min_length = min(len(df), len(predictions))
        df = df.iloc[:min_length].copy()
        predictions = predictions[:min_length]
        print(f"   → 길이를 {min_length}로 조정")
    
    df[target_col] = predictions.astype(int)
    
    # 예측 결과 요약
    value_counts = pd.Series(predictions).value_counts().sort_index()
    total = len(predictions)
    
    print(f"   ✅ {target_col} ({model_type}) 할당 완료:")
    for value, count in value_counts.items():
        percentage = (count / total) * 100
        print(f"      클래스 {value}: {count:,}개 ({percentage:.1f}%)")
    
    return df

# S1 다중분류 예측 결과 할당
if s1_success and 'S1' in ensemble_predictions:
    print(f"\n🔄 S1 다중분류 예측 결과 할당 중...")
    s1_predictions = ensemble_predictions['S1']
    submission_ensemble = assign_predictions_safely(
        submission_ensemble, 'S1', s1_predictions, "다중분류"
    )
else:
    print(f"\n❌ S1 다중분류 예측 결과가 없습니다. 기본값(0)으로 설정합니다.")
    submission_ensemble['S1'] = 0

# 이진분류 예측 결과 할당
print(f"\n🔄 이진분류 예측 결과 할당 중...")
for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
    if col in ensemble_predictions:
        predictions = ensemble_predictions[col]
        submission_ensemble = assign_predictions_safely(
            submission_ensemble, col, predictions, "이진분류"
        )
    else:
        print(f"   ⚠️  {col} 예측 결과가 없습니다. 기본값(0)으로 설정합니다.")
        submission_ensemble[col] = 0

# 최종 제출 형식 정렬
submission_ensemble = submission_ensemble[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

print(f"\n📊 최종 예측 결과 요약 (앙상블 기반):")
print("=" * 50)

for col in ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']:
    value_counts = submission_ensemble[col].value_counts().sort_index()
    total = len(submission_ensemble)
    print(f"🔹 {col}:")
    for value, count in value_counts.items():
        percentage = (count / total) * 100
        print(f"   클래스 {value}: {count:,}개 ({percentage:.1f}%)")

# 데이터 품질 검증
print(f"\n🔍 데이터 품질 검증:")
print(f"   - 총 행 수: {len(submission_ensemble):,}")
print(f"   - 결측값 확인:")
for col in ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']:
    null_count = submission_ensemble[col].isnull().sum()
    if null_count > 0:
        print(f"      {col}: {null_count}개 결측값 ⚠️")
    else:
        print(f"      {col}: 결측값 없음 ✅")

# 파일 저장
submission_path = submission_folder + submission_file
submission_ensemble.to_csv(submission_path, index=False)

print(f"\n✅ 앙상블 기반 제출 파일 저장 완료!")
print(f"   - 파일명: {submission_file}")
print(f"   - 저장 경로: {submission_path}")
print(f"   - 파일 크기: {len(submission_ensemble):,} 행 x {len(submission_ensemble.columns)} 열")

print(f"\n" + "=" * 60)
print("🎉 앙상블 모델링 프로세스 완료!")
print("=" * 60)

print(f"\n📋 완료된 작업 요약:")
print(f"   ✅ 각 모델별 상위 피처 선택 (20개씩)")
print(f"   ✅ LightGBM + CatBoost + XGBoost 개별 최적화")
print(f"   ✅ Soft Voting 앙상블 적용")
print(f"   ✅ SMOTE + GridSearchCV 하이퍼파라미터 최적화")
print(f"   ✅ 이진분류 모델 {len(successful_models)}개 학습")
print(f"   ✅ S1 다중분류 모델 학습")
print(f"   ✅ 최종 제출 파일 생성")

print(f"\n🔍 앙상블 성능 개선 포인트:")
print(f"   - 3개 알고리즘 다양성으로 편향 감소")
print(f"   - Soft Voting으로 확률 기반 예측")
print(f"   - 각 모델별 개별 하이퍼파라미터 최적화")
print(f"   - 클래스 불균형 해결 (SMOTE)")
print(f"   - 모델별 상위 특성만 사용하여 효율성 극대화")

print("=" * 60)